In [1]:
import gpxpy
import gpxpy.gpx 
import glob
from shapely.geometry import Polygon
import geopandas as gpd
import pandas as pd
import folium

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def process_gpx_to_df(file_name):
    gpx = gpxpy.parse(open(file_name, encoding="utf8")) 
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                route_info.append({
                'latitude': point.latitude,
                'longitude': point.longitude,
                'elevation': point.elevation,
                'track':track.name
            })
    gpx_df = pd.DataFrame(route_info)
    return gpx_df

In [4]:
my_dir = r"D:\Repositories\GPXr\Data"

In [5]:
gpx_dir = my_dir
gpx_list = glob.glob(gpx_dir + "/**/*.gpx", recursive = True)

In [6]:
type = 'track' # track, routes or waypoints
route_info = []
for filename in gpx_list:
    gpx_s = process_gpx_to_df(filename)
    gdf = gpd.GeoDataFrame(geometry=gpx_s.groupby(type).apply(
    lambda g: Polygon(gpd.points_from_xy(g['longitude'], g['latitude']))), crs = 'EPSG:4326')

In [7]:
gdf_point = gdf.centroid
gdf_point = pd.DataFrame(gdf_point, columns = ['geometry'])
gdf_point = gpd.GeoDataFrame(gdf_point, geometry = 'geometry', crs = 'EPSG:4326')
gdf_point["x"] = gdf_point.centroid.x
gdf_point["y"] = gdf_point.centroid.y

In [8]:
gdf.to_file('D:\Repositories\GPXr\Output\GPXr.shp')

In [9]:
# Stamen Terrain
map = folium.Map(location = [4.635950988715988, -74.08955396702645], tiles = "Stamen Terrain", zoom_start = 25)
for _, r in gdf.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    geo_j.add_to(map)
map